In [1]:
if sc:
    sc.stop()

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("22125094_lab4")
sc = SparkContext(conf = conf)

sc.getConf().getAll()

[('spark.driver.appUIAddress', 'http://edge:4051'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.port', '36599'),
 ('spark.executor.

Retrieve data.

In [3]:
raw = sc.textFile("hdfs://name:9000/Lab4_Datasets/browsing.txt")

raw.count()

31101

Question 1:

I will use SON algorithm.

Each line is a basket, and since we will work beginning with counting on basket, we will use map to separate them. Also, i will divide into 8 partitions, so that will be about 4000 baskets each. Initially, i will set contant values. Then, i will partition the raw data. We will also retrieve each partitions' size for later step of calculating local support of each partition. 

In [4]:
numPartitions = 8
s = 100
totalBaskets = raw.count()

In [5]:
data = raw.repartition(numPartitions).map(lambda line: line.split())

partitions_size = data.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()
partitions_size

[3880, 3880, 3900, 3897, 3890, 3890, 3884, 3880]

In this step, i will calculate local support value of each partition.

In [6]:
local_supports = []

for size in partitions_size:
    local_supports.append(max(1, int(s * (size / totalBaskets))))
    
local_supports

[12, 12, 12, 12, 12, 12, 12, 12]

Now we will do phase 1. Since the function now is not in Spark level (if im not mistaken), now we can only use local level Python operations. I will use defaultdict() to calculate key-value pair.

In [7]:
from itertools import combinations
from collections import defaultdict

def phase1_mapping(idx, it):
    baskets = list(it)
    sp = local_supports[idx]
    
    c2 = defaultdict(int)
    c3 = defaultdict(int)
    for basket in baskets:
        # this step is to prevent cases like pairs (1, 2) and (2, 1). It is same key, 
        # but if not sort it might return different value
        items = sorted(basket)
        for pair in combinations(items, 2):
            c2[pair] += 1
        for triple in combinations(items, 3):
            c3[triple] += 1
    
    for pair, cnt in c2.items():
        if cnt >= sp:
            yield (pair, 1)
    for triple, cnt in c3.items():
        if cnt >= sp:
            yield (triple, 1)

In [8]:
candidates = data.mapPartitionsWithIndex(phase1_mapping).reduceByKey(lambda a, b: 1).map(lambda x: x[0]).collect()

candidates

[('DAI62779', 'SNA45677'),
 ('ELE25077', 'GRO89004'),
 ('ELE17451', 'FRO79022'),
 ('ELE66600', 'SNA45677'),
 ('ELE32164', 'ELE66810'),
 ('ELE17451', 'GRO99222'),
 ('ELE74009', 'SNA38068'),
 ('DAI75645', 'DAI93865'),
 ('GRO32086', 'SNA80324'),
 ('DAI62779', 'FRO40251'),
 ('DAI33336', 'FRO40251'),
 ('DAI42493', 'ELE17451'),
 ('FRO78087', 'FRO80039'),
 ('DAI75645', 'SNA96271'),
 ('ELE74009', 'SNA80324'),
 ('FRO40251', 'GRO15017'),
 ('DAI62779', 'SNA55762'),
 ('DAI62779', 'FRO35904'),
 ('DAI88807', 'SNA59903'),
 ('SNA40784', 'SNA99873'),
 ('DAI62779', 'ELE34057'),
 ('GRO21487', 'SNA45677'),
 ('DAI62779', 'ELE14480'),
 ('DAI62779', 'DAI94679'),
 ('FRO16142', 'GRO71621'),
 ('DAI31081', 'DAI62779'),
 ('DAI62779', 'GRO56989'),
 ('ELE26917', 'FRO98729'),
 ('DAI31081', 'ELE66600'),
 ('DAI62779', 'GRO38983'),
 ('DAI62779', 'SNA96466'),
 ('DAI62779', 'FRO92469'),
 ('DAI55148', 'DAI88079'),
 ('DAI88079', 'GRO73461'),
 ('GRO73461', 'SNA80324'),
 ('ELE11111', 'SNA80324'),
 ('DAI62779', 'SNA20554'),
 

Now we will do phase 2.

In [9]:
def phase2_mapping(it):
    baskets = list(it)
    local_counts = defaultdict(int)

    for basket in baskets:
        # use set because issubset() is function for set 
        basket_set = set(basket)
        for cand in candidates:
            if set(cand).issubset(basket_set):
                local_counts[cand] += 1

    for cand, cnt in local_counts.items():
        if cnt > 0:
            yield (cand, cnt)

This is result of question 1:

In [10]:
result = data.mapPartitions(phase2_mapping).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] >= s).collect()

result

[(('DAI62779', 'SNA45677'), 604),
 (('ELE25077', 'GRO89004'), 215),
 (('ELE66600', 'SNA45677'), 154),
 (('ELE32164', 'ELE66810'), 186),
 (('ELE17451', 'GRO99222'), 148),
 (('DAI55148', 'FRO40251', 'SNA80324'), 123),
 (('DAI62779', 'FRO40251'), 1070),
 (('SNA44190', 'SNA80324'), 112),
 (('DAI62779', 'ELE17451', 'FRO40251'), 406),
 (('ELE17451', 'SNA59903', 'SNA72163'), 127),
 (('DAI62779', 'ELE26917', 'FRO40251'), 109),
 (('DAI42493', 'ELE17451'), 192),
 (('DAI75645', 'FRO40251', 'SNA80324'), 550),
 (('FRO78087', 'FRO80039'), 116),
 (('DAI75645', 'SNA96271'), 175),
 (('ELE74009', 'SNA80324'), 224),
 (('FRO40251', 'GRO15017'), 197),
 (('DAI62779', 'SNA55762'), 593),
 (('DAI62779', 'FRO35904'), 291),
 (('DAI85309', 'ELE99737', 'FRO19221'), 111),
 (('DAI62779', 'GRO17075'), 100),
 (('DAI88807', 'SNA59903'), 328),
 (('SNA40784', 'SNA99873'), 129),
 (('DAI62779', 'ELE34057'), 224),
 (('GRO21487', 'SNA45677'), 194),
 (('DAI62779', 'ELE14480'), 230),
 (('DAI62779', 'DAI94679'), 118),
 (('DAI31

Question 2:

From the result in question 1, we will store triples in dictionary.

In [11]:
triples_dict = {}
pairs_dict = {}

for tup, cnt in result:
    if len(tup) == 2:
        pairs_dict[tup] = cnt
    else:
        triples_dict[tup] = cnt

Computing Confidence for Association Rules

After identifying all frequent 3-itemsets, for each sets, we have three association rules:

- (X, Y) &#8594; Z
- (X, Z) &#8594; Y
- (Y, Z) &#8594; X

Confidence calculating function:

$\text{confidence}(X \to Y) = \frac{\text{support}(X \cup Y)}{\text{support}(X)}$

We need to apply into all 3 rules. After computing confidence for all rules, we sort them in decreasing order and
select the top 5 rules with the highest confidence.

In [12]:
rules = []

for triple, sup_xyz in triples_dict.items():
    x, y, z = triple
    
    sup_xy = pairs_dict.get((x, y), 0)
    sup_xz = pairs_dict.get((x, z), 0)
    sup_yz = pairs_dict.get((y, z), 0)
    
    if sup_xy > 0:
        rules.append((f"({x}, {y}) -> {z}", sup_xyz / sup_xy))
    if sup_xz > 0:
        rules.append((f"({x}, {z}) -> {y}", sup_xyz / sup_xz))
    if sup_yz > 0:
        rules.append((f"({y}, {z}) -> {x}", sup_xyz / sup_yz))
        
rules_sorted = sorted(rules, key=lambda x: x[1], reverse=True)

for rule, conf in rules_sorted[:5]:
    print(f"{rule}  (confidence={conf:.4f})")

(ELE17451, GRO85051) -> FRO40251  (confidence=1.0000)
(ELE26917, GRO85051) -> FRO40251  (confidence=1.0000)
(DAI55911, GRO85051) -> FRO40251  (confidence=1.0000)
(GRO21487, GRO85051) -> FRO40251  (confidence=1.0000)
(GRO73461, GRO85051) -> FRO40251  (confidence=1.0000)
